In [5]:
import numpy as np
import itertools
from numpy import linalg as LA

def cartesian(x,y):
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
    
x = np.asarray([1,2,3])
cartesian(x,x)

def permutations(x):
    return list(itertools.combinations(x, 2))

permutations(x)

[(1, 2), (1, 3), (2, 3)]

In [6]:
print(x)
np.minimum(x,2)


[1 2 3]


array([1, 2, 2])

In [7]:
x = np.asarray([1,2])
x=x/np.linalg.norm(x)
y=-x+.01
y=y/np.linalg.norm(y)
print(x,y)

[0.4472136  0.89442719] [-0.44315465 -0.89644518]


In [8]:
z=y*.2+x*.8
z=z/np.linalg.norm(z)
print(z)

[0.44856453 0.89375045]


In [9]:
print(x,y)

[0.4472136  0.89442719] [-0.44315465 -0.89644518]


In [10]:
def get_angle(vector1=None,vector2=None):
    if vector1 is None:
        vector1 = np.asarray([1,0])
    sign = np.sign(np.cross(vector1, vector2))
    angle = sign*np.arccos(np.dot(vector1, vector2) /
                          (np.linalg.norm(vector1) * np.linalg.norm(vector2)+1e-6))
    return angle

get_angle([-1,1],[1,-1])
np.sign([-1,1])==np.sign([1,-1])

array([False, False])

In [11]:
basis = [1,0]
rotation_matrix = np.asarray([[0,1],[-1,0]])
rotated_basis = np.matmul(basis, rotation_matrix)
print(rotated_basis)

[0 1]


In [12]:
import math
def get_angle(vector1=None,vector2=None):
    if vector1 is None:
        vector1 = np.asarray([1,0])
    sign = np.sign(np.cross(vector1, vector2))
    angle = sign*np.arccos(np.dot(vector1, vector2) /
                          (np.linalg.norm(vector1) * np.linalg.norm(vector2)+1e-6))
    if angle==0 and (np.sign(vector1)!=np.sign(vector2)).any():
        angle=math.pi
    return angle

def get_vector_from_angle(angle,basis=[1,0]):
    rotation_matrix = np.asarray([[0,-1],[1,0]])
    rotated_basis = np.matmul(rotation_matrix,basis.transpose())
    transform=np.asarray([basis,rotated_basis]).transpose()
    print(transform)
    return np.matmul(transform,np.asarray([math.tan(angle),1]))

x_angles = np.asarray([0,1])
t_angles = np.asarray([1,-1])
tau_turning_rate = .1
angle = get_angle(x_angles,t_angles)
print(x_angles, t_angles, angle)

convex_rate = min(tau_turning_rate/abs(angle),1) # always between 0 and 1            
print("Rate",convex_rate)
x_angles = get_vector_from_angle(convex_rate*angle*0, basis=x_angles)
print(x_angles)
angle = get_angle(x_angles,t_angles)
print(angle)

[0 1] [ 1 -1] -2.3561937830863138
Rate 0.042441330894699474
[[ 0 -1]
 [ 1  0]]
[-1.  0.]
2.3561937830863138


In [13]:
x_angles = np.asarray([0,1])
t_angles = np.asarray([1,-1])

x_angles = np.asarray([-1,1])
t_angles = np.asarray([1,-1])


tau_turning_rate = 1

def get_vector_from_angle(angle):
    return np.asarray([math.tan(angle),1])

old_angle =get_angle(None,x_angles)
new_angle =get_angle(None,t_angles)

angle = get_angle(x_angles,t_angles)

if old_angle < -math.pi/2:
    old_angle+=2*math.pi
if new_angle < -math.pi/2:
    new_angle+=2*math.pi

convex_rate = min(tau_turning_rate/abs(angle),1) # always between 0 and 1            
final_angle = (convex_rate*new_angle + (1-convex_rate)*old_angle)
x_angles=get_vector_from_angle(final_angle)
print(old_angle)
print(new_angle)
print(final_angle)
#print(convex_rate, angle)
assert(old_angle-new_angle)<=math.pi




2.3561937830863138
-0.7853988705034795
1.3561937830863136


In [14]:
m = get_angle([1,0], [1,1])
n = get_angle([1,0], [-1,-1])
print(m,n)

0.7853988705034795 -2.3561937830863138


In [15]:
get_vector_from_angle(.8)

array([1.02963856, 1.        ])

In [50]:
def get_angle(vector1=None,vector2=None):
    if vector1 is None:
        vector1 = np.asarray([1,0])
    sign = np.sign(np.cross(vector1, vector2))
    angle = sign*np.arccos(np.dot(vector1, vector2) /
                          (np.linalg.norm(vector1) * np.linalg.norm(vector2)+1e-6))
    if angle==0 and (np.sign(vector1)!=np.sign(vector2)).any():
        angle=math.pi
    return angle

def unit_vector(vector):
    return vector/LA.norm(vector)

def rotate(x_angles, t_angles, turning_rate, tau=.1):
    # Turning rate: turning rate * tau (timestep) - can we scale this to be from 0 to 1?
    tau_turning_rate = tau * turning_rate

    for i in range(x_angles.shape[0]):

        # angle from direction to target direction -pi to pi
        angle = get_angle(x_angles[i], t_angles[i])

        # if vectors are completely opposite
        if abs(angle)<1e-6 and (np.sign(x_angles[i]) != np.sign(t_angles[i])).any():
            angle=math.pi

        if abs(angle) <= tau_turning_rate:
            x_angles[i] = t_angles[i]
        elif abs(angle) > tau_turning_rate:
            old_angle =get_angle(None,x_angles[i])
            print("Angle", angle)
            c = min(1,tau_turning_rate/abs(angle))
            t = old_angle+angle*c
            print("Current Angle",old_angle)
            print("T",t)
            print("C",c)
            new_vector = math.cos(t), math.sin(t)
            x_angles[i]=new_vector
            #print("UGH",x_angles[i], new_vector)

    return x_angles

x = [1.0,0]
t = [-1.0,1]
x_angles=np.asarray([x])
t_angles=[t]
for i in range(0,8):
    print("Current trajectory",x_angles)
    x_angles = rotate(x_angles, t_angles, turning_rate=5)

Current trajectory [[1. 0.]]
Angle 2.3561937830863138
Current Angle 0.0
T 0.5
C 0.21220665447349737
Current trajectory [[0.87758256 0.47942554]]
Angle 1.8561942827217164
Current Angle 0.5000018304828242
T 1.0000018304828242
C 0.2693683547321651
Current trajectory [[0.54030077 0.84147197]]
Angle 1.3561928138304145
Current Angle 1.0000024725720804
T 1.5000024725720804
C 0.368679139795621
Current trajectory [[0.07073474 0.99749516]]
Angle 0.8561926310817919
Current Angle 1.5000025434843687
T 2.000002543484369
C 0.583980732663226
Current trajectory [[-0.41614915  0.90929637]]
Current trajectory [[-1.  1.]]
Current trajectory [[-1.  1.]]
Current trajectory [[-1.  1.]]


In [ ]:

def get_angle(vector1=None,vector2=None):
    if vector1 is None:
        vector1 = np.asarray([1,0])
    sign = np.sign(np.cross(vector1, vector2))
    angle = sign*np.arccos(np.dot(vector1, vector2) /
                          (np.linalg.norm(vector1) * np.linalg.norm(vector2)+1e-6))
    if angle==0 and (np.sign(vector1)!=np.sign(vector2)).any():
        angle=math.pi
    return angle

def unit_vector(vector):
    return vector/LA.norm(vector)

def rotate(x_angles, t_angles, turning_rate, tau=.1):
    # Turning rate: turning rate * tau (timestep) - can we scale this to be from 0 to 1?
    tau_turning_rate = tau * turning_rate

    for i in range(x_angles.shape[0]):

        # angle from direction to target direction -pi to pi
        angle = get_angle(x_angles[i], t_angles[i])

        # if vectors are completely opposite
        if abs(angle)<1e-6 and (np.sign(x_angles[i]) != np.sign(t_angles[i])).any():
            angle=math.pi

        if abs(angle) <= tau_turning_rate:
            x_angles[i] = t_angles[i]
        elif abs(angle) > tau_turning_rate:
            old_angle =get_angle(None,x_angles[i])
            c = min(1,tau_turning_rate/abs(angle))
            t = old_angle+angle*c
            new_vector = math.cos(t), math.sin(t)
            x_angles[i]=new_vector

    return x_angles